In [5]:
import pandas as pd
import numpy as np
import pyterrier_deepct
import pyterrier as pt
if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

In [6]:
from xml.etree import ElementTree
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics2020.xml')
data = tree.getroot()
titles20 = []
titles21 = []
title_ids20 = []
title_ids21 = []
nums20 = []
nums21 = []
for topic in data:
    titles20.append(topic.find('title').text.lower()[:-1])
    num20 = topic.find('number').text
    nums20.append(num20)
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics.xml')
data = tree.getroot()    
for topic in data:
    titles21.append(topic.find('title').text.lower()[:-1])
    num21 = topic.find('number').text
    nums21.append(num21)
topics = pd.DataFrame()
topics['qid'] = nums20 + nums21
topics['query'] = titles20 + titles21

topics21 = pd.DataFrame()
topics21['qid'] = nums21
topics21['query'] = titles21

topics20 = pd.DataFrame()
topics20['qid'] = nums20
topics20['query'] = titles20

qrels_path=("/workspace/src/evaluate_targers/new_qrels.qrels")
qrels = pt.io.read_qrels(qrels_path)

qrels21_path=("/workspace/src/evaluate_targers/qrels.qrels")
qrels21 = pt.io.read_qrels(qrels21_path)

qrels20_path=("/workspace/src/evaluate_targers/qrels20.qrels")
qrels20 = pt.io.read_qrels(qrels20_path)

docnos = qrels.docno

In [7]:
import os
# rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/"
rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_original-100"

objective="ndcg_cut_5"
for i, (subdir, dirs, files) in enumerate(os.walk(rootdir)):
    for file in files:
        if(file == 'data.properties'):
            path = subdir + '/'+ file
            print('\n' + subdir)
            indexref = pt.IndexRef.of(path)
            index = pt.IndexFactory.of(indexref)
#             bm25_rm3 = (pt.BatchRetrieve(indexref, wmodel="BM25") >> 
#             pt.rewrite.RM3(indexref) >> 
#             pt.BatchRetrieve(indexref, wmodel="BM25"))
            bm25_for_qe = pt.BatchRetrieve(index, wmodel="BM25")
            rm3 = pt.rewrite.RM3(index, fb_terms=10, fb_docs=3)
            pipe_qe = bm25_for_qe >> rm3 >> bm25_for_qe
            
            param_map = {
                rm3 : {
                    "fb_terms" : list(range(4, 16, 2)),
                    "fb_docs" : list(range(4, 10, 2)),
                    "fb_lambda": [0.2,0.4,0.6,0.8,1.0]
                }
            }
            
            pipe_qe, parameters = pt.KFoldGridSearch(
                pipe_qe,
                param_map,
                [topics20, topics21],
                [qrels20, qrels21],
                objective
            )
            print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos)]], topics, qrels, 
                                names=['tuned_bm25_rm3_unjudged'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]))
            print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos)]], topics20, qrels, 
                                names=['tuned_bm25_rm3_unjudged20'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]))
            print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos)]], topics21, qrels, 
                                names=['tuned_bm25_rm3_unjudged21'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]))            
            



/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_original-100
Fold 1
Best ndcg_cut_5 is 0.584666
Best setting is ['<pyterrier.rewrite.RM3 object at 0x7f1ff9e00910> fb_terms=12', '<pyterrier.rewrite.RM3 object at 0x7f1ff9e00910> fb_docs=4', '<pyterrier.rewrite.RM3 object at 0x7f1ff9e00910> fb_lambda=0.6']
Fold 2
Best ndcg_cut_5 is 0.325742
Best setting is ['<pyterrier.rewrite.RM3 object at 0x7f1ff9e00910> fb_terms=14', '<pyterrier.rewrite.RM3 object at 0x7f1ff9e00910> fb_docs=8', '<pyterrier.rewrite.RM3 object at 0x7f1ff9e00910> fb_lambda=0.4']
                      name  ndcg_cut_5  ndcg_cut_25     bpref
0  tuned_bm25_rm3_unjudged    0.670511     0.638806  0.599561
                        name  ndcg_cut_5  ndcg_cut_25     bpref
0  tuned_bm25_rm3_unjudged20     0.69473      0.62924  0.546714
                        name  ndcg_cut_5  ndcg_cut_25     bpref
0  tuned_bm25_rm3_unjudged21    0.646776      0.64818  0.651351


In [8]:
import os
rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/"
folders = ['conclusions','pool_topics','topics','topics_conclusion','topics_conclusion_leakage','topics_leakage','pool_topics_conclusions','conclusions_leakage','pools_conclusions']
objective="ndcg_cut_5"
i = 0
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if(file == 'data.properties'):
            path = subdir + '/'+ file
            print('\n' + subdir)
            indexref = pt.IndexRef.of(path)
            index = pt.IndexFactory.of(indexref)
            bm25_for_qe = pt.BatchRetrieve(index, wmodel="BM25")
            rm3 = pt.rewrite.RM3(index, fb_terms=10, fb_docs=3)
            pipe_qe = bm25_for_qe >> rm3 >> bm25_for_qe
            
            param_map = {
                rm3 : {
                    "fb_terms" : list(range(4, 16, 2)),
                    "fb_docs" : list(range(4, 10, 2)),
                    "fb_lambda": [0.2,0.4,0.6,0.8,1.0]
                }
            }
            
            pipe_qe, parameters = pt.KFoldGridSearch(
                pipe_qe,
                param_map,
                [topics20, topics21],
                [qrels20, qrels21],
                objective
            )
            print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos)]], topics, qrels, names=[folders[i] + '_tuned_bm_rm3_unjudged'], 
                                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]))
            print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos)]], topics20, qrels, names=[folders[i] + '_tuned_bm_rm3_unjudged20'], 
                                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]))
            print(pt.Experiment([pipe_qe[pipe_qe['docno'].isin(docnos)]], topics21, qrels, names=[folders[i] + '_tuned_bm_rm3_unjudged21'], 
                                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]))            

            i+=1


/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/conclusionspd_index
Fold 1
Best ndcg_cut_5 is 0.558365
Best setting is ['<pyterrier.rewrite.RM3 object at 0x7f1ff9e2d590> fb_terms=6', '<pyterrier.rewrite.RM3 object at 0x7f1ff9e2d590> fb_docs=4', '<pyterrier.rewrite.RM3 object at 0x7f1ff9e2d590> fb_lambda=0.8']
Fold 2
Best ndcg_cut_5 is 0.339332
Best setting is ['<pyterrier.rewrite.RM3 object at 0x7f1ff9e2d590> fb_terms=6', '<pyterrier.rewrite.RM3 object at 0x7f1ff9e2d590> fb_docs=8', '<pyterrier.rewrite.RM3 object at 0x7f1ff9e2d590> fb_lambda=0.8']
                                name  ndcg_cut_5  ndcg_cut_25     bpref
0  conclusions_tuned_bm_rm3_unjudged     0.68527      0.66211  0.642888
                                  name  ndcg_cut_5  ndcg_cut_25     bpref
0  conclusions_tuned_bm_rm3_unjudged20    0.688609     0.621772  0.536259
                                  name  ndcg_cut_5  ndcg_cut_25     bpref
0  conclusions_tuned_bm_rm3_unjudge

                                        name  ndcg_cut_5  ndcg_cut_25  \
0  conclusions_leakage_tuned_bm_rm3_unjudged     0.68535     0.663139   

      bpref  
0  0.634131  
                                          name  ndcg_cut_5  ndcg_cut_25  \
0  conclusions_leakage_tuned_bm_rm3_unjudged20    0.686443     0.626767   

      bpref  
0  0.526546  
                                          name  ndcg_cut_5  ndcg_cut_25  \
0  conclusions_leakage_tuned_bm_rm3_unjudged21    0.684279     0.698784   

      bpref  
0  0.739564  

/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/pools_conclusionspd_index
Fold 1
Best ndcg_cut_5 is 0.539135
Best setting is ['<pyterrier.rewrite.RM3 object at 0x7f1ff9a25350> fb_terms=12', '<pyterrier.rewrite.RM3 object at 0x7f1ff9a25350> fb_docs=4', '<pyterrier.rewrite.RM3 object at 0x7f1ff9a25350> fb_lambda=0.8']
Fold 2
Best ndcg_cut_5 is 0.345878
Best setting is ['<pyterrier.rewrite.RM3 object at 0x7f1ff9a25350> fb